# Language Modeling: Bigram

In this project, a bigram language model will be implemented and trained on a dataset containing news headlines. We will then evaluate our model using the perplexity metric.

## Setup
Here we will be importing the necessary modules, as well as doing some basic preprocessing of the text.

In [1]:
import math
import random
import re
import string
import numpy as np

# Reading in the training and development datasets
with open("headlines.train", 'r') as f:
    headlines_train = f.readlines()
with open("headlines.dev", 'r') as f:
    headlines_dev = f.readlines()
    
# Removing excess punctuation and newline
regex = re.compile('[%s]' % re.escape(string.punctuation))
headlines_train = [regex.sub('', h.split("\n")[0]) for h in headlines_train]
headlines_dev = [regex.sub('', h.split("\n")[0]) for h in headlines_dev]

# Defining UNK, START and STOP tokens
UNK_TOKEN = "<UNK>" # UNKNOWN - mapping rare words
START_TOKEN = "<START>"
STOP_TOKEN = "<STOP>"

Before we begin, let's look at what some of the headlines look like.

In [2]:
for headline in random.sample(headlines_train, 5):
    print(headline)
    print()

lockyer valley housing market bouncing back after

fixed line death strangles telstra profits

national rural news tuesday 6th november

builders warned to control rubbish

miners fall on china slowdown speculation



## Counting Bigrams

In [3]:
def count_unigrams(text, unigram_dict):
    """
    :param text: A headline, consisting of a string of words
    :param unigram_dict: A dictionary containing unigrams as keys and their respective counts as values
    """
    tokens = [START_TOKEN] + text.split(" ") + [STOP_TOKEN]
    for i in range(len(tokens)):
        unigram = tokens[i]
        if unigram not in unigram_dict:
            unigram_dict[unigram] = 1
        else:
            unigram_dict[unigram] += 1

def count_bigrams(text, bigram_dict):
    """
    :param text: A headline, consisting of a string of words
    :param bigram_dict: A dictionary containing bigrams as keys and their respective counts as values
    """
    tokens = [START_TOKEN] + text.split(" ") + [STOP_TOKEN]
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i+1])
        if bigram not in bigram_dict:
            bigram_dict[bigram] = 1
        else:
            bigram_dict[bigram] += 1


## Computing Probability
Let's calculate the probability of seeing a word given the previous word, along with Laplace smoothing parameterized by `alpha`.

In [4]:
def get_probability(target, context, unigram_dict, bigram_dict, alpha, vocab_size):
    """
    :param target: The word whose probability of seeing is being computed given the context
    :param context: The word directly preceeding the target word
    :param unigram_dict: A dictionary containing unigrams as keys and their respective counts as values
    :param bigram_dict: A dictionary containing bigrams as keys and their respective counts as values
    :param alpha: The amount of additive smoothing being applied
    :param vocab_size: The size of the training vocabulary
    :return: The probability of seeing the target word given the context
    """
    bigram = bigram_dict[(context, target)] if (context, target) in bigram_dict else 0
    return (bigram + alpha) / (unigram_dict[context] + vocab_size*alpha)

## Word Sampling
Once we can calculate the desired probabilities, we can now use that to sample words for generation. We will use it to sample a new word in accordance with its probability of following the previous word.

In [5]:
def sample_word(words, probs):
    """
    :param words: The list of words to sample from
    :param probs: The probabilities of seeing each word; probs[i] is the probability of seeing word[i]
    :return: A word whose sampling likelihood is the probability of being seen given the context
    """
    return words[np.argmax(np.random.multinomial(1, probs, 1))]

## Generating New Headlines
Now that we've all the key parts of our language model completed, let's see how well we can generate new headlines!

In [6]:
alpha = 0.0205 # optimum for lowest perplexity
min_freq = 10 # The minimum word frequency to be present in the vocabulary

# The following are used to keep track of and remove infrequent words
low_freq = set()
all_words = {}

def generate_headline(unigram_dict, bigram_dict, alpha):
    sent = [START_TOKEN]
    while not sent[-1] == STOP_TOKEN:
        words = list(vocab)
        probs = [get_probability(word, sent[-1], unigram_dict, bigram_dict, alpha, len(vocab)) for word in words]
        next_word = sample_word(words, probs)
        sent.append(next_word)
    print(' '.join(sent[1:-1]))

def replace_text_train(text):
    return " ".join([UNK_TOKEN if t in low_freq else t for t in text.split()])

def replace_text_dev(text):
    return " ".join([UNK_TOKEN if t not in vocab else t for t in text.split()])

# Finding all words with low frequency
for h in headlines_train:
    count_unigrams(h, all_words)
for word, count in all_words.items():
    if count <= min_freq:
        low_freq.add(word)
# Replacing low frequency words from training dataset with UNK
headlines_train_clean = [replace_text_train(h) for h in headlines_train]

# Initialize unigram and bigram dictionaries
unigrams = {}
bigrams = {}

# Generating unigram and bigram dictionaries
for h in headlines_train_clean:
    count_unigrams(h, unigrams)
    count_bigrams(h, bigrams)

# Creating the training vocabulary
vocab = set([item for sublist in map(lambda x: x.split(" "), headlines_train_clean) for item in sublist])
vocab.add(START_TOKEN)
vocab.add(STOP_TOKEN)

# Replacing unseen vocabulary from development dataset with UNK
headlines_dev_clean = [replace_text_dev(h) for h in headlines_dev]

In [7]:
for _ in range(5):
    generate_headline(unigrams, bigrams, alpha)
    print()

enrolments diesel tamar prior yahoo mile manchester city

brisbane parade to take joint shelves dams

<UNK> safe in wake health australia cotton taswater uproar runaway apprenticeships usual arrested

lord edward comp christensen confirmed bionic compensation to <UNK> shooting study calls kerry locusts easier under bowlers bats alkatiri briefed changed deaths

<UNK> planning maldives female <UNK> replacement points



## Model Evaluation: Perplexity
We can see how well our model does when encountering text from an unseen development set by calculating the perplexity. The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of words. The higher the conditional probability of the word sequence, the lower the perplexity (which is what we aim for, on the test set). 

For a bigram model trained on a test set W containing N words:
$$ \begin{equation*}
    PP(W) = \sqrt[N]{ \prod_{i=1}^{N}  \frac{1}{P( w_{i} |  w_{i-1})}  } 
\end{equation*} $$

In [8]:
def calculate_perplexity(text, unigram_dict, bigram_dict, alpha, vocab_size):
    """
    :param text: A headline, consisting of a string of words
    """
    tokens = [START_TOKEN] + text.split(" ") + [STOP_TOKEN]
    log_sum = 0
    for i in range(len(tokens) - 1):
        prob = get_probability(tokens[i+1], tokens[i], unigram_dict, bigram_dict, alpha, vocab_size)
        log_sum += np.log(prob)*(-1) # using log sum to prevent integer underflow

    return np.exp(log_sum / (len(tokens) - 1))

In [9]:
perplexities = []
for h in headlines_dev_clean:
    perp = calculate_perplexity(h, unigrams, bigrams, alpha, len(vocab))
    perplexities.append(perp)
print("Average perplexity on dev set: %.02f" % (sum(perplexities) / len(perplexities)))

Average perplexity on dev set: 782.77
